<a href="https://colab.research.google.com/github/DiMorten/osss-mcr/blob/coords7/train_and_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [1]:
!pip install icecream
#%tensorflow_version 1.x
import os
!pip install kora
from kora import drive
import time
!pip install colorama
!pip install unrar

ds_path='/content/drive/My Drive/PhD/datasets/lm_data/'

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
git_clone = True

if git_clone == True:
  os.chdir('/content')
  %rm -rf osss-mcr
  !git clone --branch coords7 https://github.com/DiMorten/osss-mcr.git

Cloning into 'osss-mcr'...
remote: Enumerating objects: 2539, done.
remote: Counting objects: 100% (338/338), done.
remote: Compressing objects: 100% (251/251), done.
remote: Total 2539 (delta 230), reused 153 (delta 62), pack-reused 2201
Receiving objects: 100% (2539/2539), 37.67 MiB | 17.38 MiB/s, done.
Resolving deltas: 100% (1696/1696), done.


## Download images into proper folder

In [4]:
# !cp -r /content/drive/MyDrive/PhD/datasets/lm_data /content/osss-mcr/dataset/

os.chdir('/content/osss-mcr/dataset/')
!gdown --id 1MPqRKFcT2GzWGQM7PO7vy8X-smf5kFAV

Downloading...
From: https://drive.google.com/uc?id=1MPqRKFcT2GzWGQM7PO7vy8X-smf5kFAV
To: /content/osss-mcr/dataset/lm_data.rar
7.55GB [01:17, 97.4MB/s]


In [5]:
!unrar x -y "lm_data.rar" "/content/osss-mcr/dataset/"
os.remove("lm_data.rar")


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from lm_data.rar

Extracting  /content/osss-mcr/dataset/lm_data/coords_test.npy              0%  OK 
Extracting  /content/osss-mcr/dataset/lm_data/coords_test_centered.npy       0%  OK 
Extracting  /content/osss-mcr/dataset/lm_data/coords_train.npy             0%  OK 
Extracting  /content/osss-mcr/dataset/lm_data/coords_train_centered.npy       0%  OK 
Creating    /content/osss-mcr/dataset/lm_data/in_sar                  OK
Extracting  /content/osss-mcr/dataset/lm_data/in_sar/20160927_S1.npy       0%  1%  2%  OK 
Extracting  /content/osss-mcr/dataset/lm_data/in_sar/20161015_S1.npy       2%  3%  4%  5%  OK 
Extracting  /content/osss-mcr/dataset/lm_data/in_sar/20161120_S1.npy       5%  6%  7%  OK 
Extracting  /content/osss-mcr/dataset/lm_data/in_sar/20161214_S1.npy       7%  8%  9% 10%  OK 
Extracti

In [6]:
os.chdir('/content/osss-mcr/')
os.getcwd()
os.listdir()

['obj',
 'evaluate_open_set.py',
 'parameters',
 'README.md',
 'train_and_evaluate_open_set.ipynb',
 '__init__.py',
 '.gitignore',
 'dataset',
 'train_and_evaluate_openset.ipynb',
 '.git',
 'evaluate.py',
 'deb.py',
 'train_and_evaluate.ipynb',
 'environment.yml',
 'train_and_evaluate_open_set.py',
 'train_and_evaluate.py',
 'src']

In [7]:
from colorama import init
init()
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout, Conv2DTranspose
# from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping
from tensorflow.keras.optimizers import Adam,Adagrad 
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import tensorflow.keras as keras

import numpy as np
from sklearn.utils import shuffle
import cv2
import argparse
import tensorflow as tf

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import metrics
import sys
import glob

from sklearn.metrics import confusion_matrix,f1_score,accuracy_score,classification_report
# Local
from src.densnet import DenseNetFCN
from src.densnet_timedistributed import DenseNetFCNTimeDistributed

#from metrics import fmeasure,categorical_accuracy
import deb
from src.keras_weighted_categorical_crossentropy import weighted_categorical_crossentropy, sparse_accuracy_ignoring_last_label, weighted_categorical_crossentropy_ignoring_last_label, categorical_focal_ignoring_last_label, weighted_categorical_focal_ignoring_last_label
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import ConvLSTM2D, UpSampling2D, multiply
from tensorflow.keras.regularizers import l1,l2
import time
import pickle
#from tensorflow.keras_self_attention import SeqSelfAttention
import pdb
import pathlib
from pathlib import Path, PureWindowsPath
from tensorflow.keras.layers import Conv3DTranspose, Conv3D

from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from collections import Counter


from src.generator import DataGenerator, DataGeneratorWithCoords, DataGeneratorWithCoordsRandom

import matplotlib.pyplot as plt
sys.path.append('../../../dataset/dataset/patches_extract_script/')
from src.dataSource import DataSource, SARSource, Dataset, LEM, LEM2, CampoVerde
from src.model_input_mode import MIMFixed, MIMVarLabel, MIMVarSeqLabel, MIMVarLabel_PaddedSeq, MIMFixedLabelAllLabels, MIMFixed_PaddedSeq
from parameters.params_train import ParamsTrain
from parameters.params_mosaic import ParamsReconstruct

from icecream import ic
from src.monitor import Monitor, MonitorNPY, MonitorGenerator, MonitorNPYAndGenerator
import natsort
from src.model import NetModel, ModelFit, ModelLoadGeneratorWithCoords
from src.dataset import Dataset, DatasetWithCoords

from src.patch_extractor import PatchExtractor

from src.mosaic import seq_add_padding, add_padding, Mosaic, MosaicHighRAM, MosaicHighRAMPostProcessing
from src.postprocessing import PostProcessingMosaic

from src.metrics import Metrics, MetricsTranslated

ic.configureOutput(includeContext=True)
np.random.seed(2021)
tf.random.set_seed(2021)
tf.compat.v1.disable_eager_execution()
tf.compat.v1.experimental.output_all_intermediates(True)

from train_and_evaluate import TrainTest

In [8]:
!nvidia-smi

Wed Sep  8 23:06:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Set parameters

Parameters can be modified in /content/osss-mcr/networks/convlstm_networks/train_src/parameters/params_train.py

In [9]:
paramsTrainCustom = {
	'getFullIms': True,
	'coordsExtract': True,
	'train': True,
	'openSetMethod': None, # Options: None, OpenPCS, OpenPCS++
#		'openSetLoadModel': True,
	'selectMainClasses': True,
	'dataset': 'lm', # lm: L Eduardo Magalhaes.
	'seq_date': 'mar',	# jun, mar	
    'dataSource': SARSource()
}

paramsTrain = ParamsTrain('parameters/', **paramsTrainCustom)

paramsTrain.dataSource = SARSource()

trainTest = TrainTest(paramsTrain)



[@debug] params_train.py:107 in __init__()- self.seq_date: 'mar'


self.known_classes [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]


[@debug] params_train.py:166 in __init__()- self.stride: 32


['no_mode.json', 'parameters_openset.json', 'params_batchprocessing.py', '__init__.py', '__pycache__', 'parameters_closedset_groupclasses.json', 'params_mosaic.py', 'save_nonaugmented_train_patches.json', 'params_train.py']
[@debug] self.seq_mode = fixed
[@debug] self.mim = <model_input_mode.MIMFixed_PaddedSeq object at 0x7f8c91e1bb50>
[@debug] self.ds = <src.dataSource.LEM object at 0x7f8c91e88250>
[@debug] dataSource.name = SARSource
self.im_list ['20170413_S1', '20170519_S1', '20170612_S1', '20170706_S1', '20170811_S1', '20170916_S1', '20171010_S1', '20171115_S1', '20171209_S1', '20180114_S1', '20180219_S1', '20180315_S1']
fixed mar
[@debug] self.t_len = 12
20170413
20170519
20170612
20170706
20170811
20170916
20171010
20171115
20171209
20180114
20180219
20180315
dotys_sin_cos.shape (12, 2)
[103, 139, 163, 187, 223, 259, 283, 319, 343, 14, 50, 74]
[[0.9917   0.4104  ]
 [0.849    0.142   ]
 [0.6763   0.03214 ]
 [0.4744   0.000663]
 [0.1897   0.1079  ]
 [0.01993  0.3604  ]
 [0.00414  

In [ ]:
trainTest.main()

[@debug] patch_extractor.py:17 in __init__()
         self.dataSource: <src.dataSource.SARSource object at 0x7f8c92ce1b10>
[@debug] patch_extractor.py:26 in __init__()
         self.conf['path']/self.label_folder/"/": PosixPath('/')
[@debug] patch_extractor.py:35 in __init__()
         self.conf["in_npy_path"]: PosixPath('dataset/lm_data/in_sar')
[@debug] patch_extractor.py:43 in __init__()
         self.conf["train"]["mask"]["dir"]: PosixPath('dataset/lm_data/TrainTestMask.tif')
[@debug] patch_extractor.py:44 in __init__()
         os.getcwd(): '/content/osss-mcr'
[@debug] patch_extractor.py:90 in getFullIms()
         patch["full_ims"].shape: (12, 8484, 8658, 2)
[@debug] patch_extractor.py:91 in getFullIms()
         self.dataset.im_list: ['20170413_S1',
                                '20170519_S1',
                                '20170612_S1',
                                '20170706_S1',
                                '20170811_S1',
                                '20170916_S1'

0 0
[@debug] conf["in_npy_path"]/(im_names[t_step]+".npy") = dataset/lm_data/in_sar/20170413_S1.npy
[@debug] patch["full_ims"].dtype = float16
[@debug] np.average(patch["full_ims"][t_step]) = 0.136962890625
[@debug] np.max(patch["full_ims"][t_step]) = 133.125
[@debug] np.min(patch["full_ims"][t_step]) = 0.0
dataset/lm_data/labels/20170413_S1.tif
[@debug] conf["path"]/(self.dataSource.label_folder+"/"+label_names[t_step]+".tif") = dataset/lm_data/labels/20170413_S1.tif
[@debug] np.unique(patch["full_label_ims"][t_step],return_counts=True) = (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13], dtype=int8), array([67849305,    18593,   268963,    84263,    45419,   106962,
         286551,  1325915,    21332,   208028,    84083,   770815,
          38849,  2345394]))
1 0
[@debug] conf["in_npy_path"]/(im_names[t_step]+".npy") = dataset/lm_data/in_sar/20170519_S1.npy
[@debug] patch["full_ims"].dtype = float16
[@debug] np.average(patch["full_ims"][t_step]) = 0.158203125
[@debug] 

[@debug] dataSource.py:254 in im_load()
         patch["full_ims"].shape: (12, 8484, 8658, 2)
[@debug] dataSource.py:255 in im_load()
         patch["full_label_ims"].shape: (12, 8484, 8658)
[@debug] dataSource.py:256 in im_load()
         patch["full_ims"].dtype: dtype('float16')
[@debug] dataSource.py:257 in im_load()
         patch["full_label_ims"].dtype: dtype('int8')


[@debug] np.unique(patch['full_label_ims'],return_counts=True) = (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
      dtype=int8), array([814191660,  10948507,   2561907,    665775,    528718,    377736,
         1238182,   5116663,    255984,   1996951,    609668,   9000640,
          711710,  32409149,    840414]))
-1.0 1.0 0.1404
[@debug] self.dataset.name = lm
[@debug] self.dataset.scaler_name = lm
[@debug] self.dataset.seq_mode = fixed
[@debug] self.dataset.seq_date = mar
[@debug] self.dataset.scaler_load = False
[@debug] im.shape = (12, 8484, 8658, 2)
[@debug] im_flat[mask_flat==1,:].shape = (51633252, 2)
0.0001221 1.0 0.0494
[@debug] im_norm.shape = (12, 8484, 8658, 2)
FINISHED NORMALIZING, RESULT:
-60.66 58.44 3.367
[@debug] im.shape = (12, 8484, 8658)
[@debug] mask_train.shape = (8484, 8658)
[@debug] im.dtype = uint8
[@debug] mask_train.dtype = uint8
[@debug] im_train.shape = (12, 8484, 8658)
Train masked unique/count [ 0  1  2  3  4  5  6  7  8  9 10 11 

[@debug] patch_extractor.py:127 in getFullIms()
         self.paramsTrain.path / 'full_ims/full_ims_test.npy': PosixPath('dataset/lm_data/full_ims/full_ims_test.npy')


...

 — val_f1: [59.24 19.78  3.63 56.87]
 — val_precision: [76.96 12.5  83.13 41.27]
 — val_recall: [48.15 47.4   1.85 91.48]
 — mean_f1: 34.88
oa 46.13
Found best weights at epoch 1
175/175 [==============================] - 53s 174ms/step - batch: 87.0000 - size: 16.0000 - loss: 0.1088 - accuracy: 0.3639 - val_loss: 0.1058 - val_accuracy: 0.4078 - mean_f1: 34.8800
Epoch 2/70
175/175 [==============================] - ETA: 0s - batch: 87.0000 - size: 16.0000 - loss: 0.0701 - accuracy: 0.2422

[@debug] generator.py:192 in __len__()- n_batches: 69
[@debug] generator.py:192 in __len__()- n_batches: 69
[@debug] generator.py:192 in __len__()- n_batches: 69


 — val_f1: [72.87 35.95 36.02 67.06]
 — val_precision: [92.87 22.35 99.98 54.47]
 — val_recall: [59.96 91.86 21.97 87.22]
 — mean_f1: 52.975
oa 60.25
Found best weights at epoch 2
175/175 [==============================] - 28s 162ms/step - batch: 87.0000 - size: 16.0000 - loss: 0.0701 - accuracy: 0.2422 - val_loss: 0.0847 - val_accuracy: 0.3812 - mean_f1: 52.9750
Epoch 3/70
175/175 [==============================] - ETA: 0s - batch: 87.0000 - size: 16.0000 - loss: 0.0566 - accuracy: 0.2634

[@debug] generator.py:192 in __len__()- n_batches: 69
[@debug] generator.py:192 in __len__()- n_batches: 69
[@debug] generator.py:192 in __len__()- n_batches: 69


 — val_f1: [93.74 68.8  97.75 81.74]
 — val_precision: [94.39 71.15 99.91 75.43]
 — val_recall: [93.1  66.61 95.68 89.18]
 — mean_f1: 85.5075
oa 90.68
Found best weights at epoch 3
175/175 [==============================] - 29s 166ms/step - batch: 87.0000 - size: 16.0000 - loss: 0.0566 - accuracy: 0.2634 - val_loss: 0.0487 - val_accuracy: 0.6025 - mean_f1: 85.5075
Epoch 4/70
141/175 [=======================>......] - ETA: 4s - batch: 70.0000 - size: 16.0000 - loss: 0.0511 - accuracy: 0.2763